In [1]:
! pip install mlxtend

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler,FunctionTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [91]:
df_train=pd.read_csv("train.csv")

In [92]:
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [93]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [94]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

In [97]:
# Columns which have zero values in more than 50% rows are removed
def column_with_most_reduntant_values(reduntant_value,df_train:pd.DataFrame,threshold:float):
    reduntant_counts={}
    columns_with_most_reduntant_counts=[]
    for col in df_train.columns:
        if df_train[col].dtypes=='int' or df_train[col].dtypes=='float':
            for values in df_train[col]:
                if values==reduntant_value:
                    reduntant_counts[col]=1+reduntant_counts.get(col,0)
    for col in reduntant_counts:
        if reduntant_counts[col]>=(threshold*len(df_train)):
            columns_with_most_reduntant_counts.append(col)
    df_train.drop(columns_with_most_reduntant_counts,axis=1,inplace=True)

In [98]:
column_with_most_reduntant_values(reduntant_value=0,df_train=df_train,threshold=0.5)

In [100]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,GarageCond,PavedDrive,OpenPorchSF,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,61,NaN,NaN,NaN,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,TA,Y,0,NaN,NaN,NaN,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,TA,Y,42,NaN,NaN,NaN,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,TA,Y,35,NaN,NaN,NaN,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,TA,Y,84,NaN,NaN,NaN,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,40,NaN,NaN,NaN,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,MnPrv,NaN,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,60,NaN,GdPrv,Shed,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,NaN,NaN,4,2010,WD,Normal


In [104]:

def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [105]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [106]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [107]:
df_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'OpenPorchSF', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition'],
      dtype='object')

In [108]:
i=0
for cols in df_train.columns:
    if df_train[cols].dtype=="int64":
        print(cols,i)
#         i+=1
    elif df_train[cols].dtype=="object":
#         print(cols,i)
        i+=1
    elif df_train[cols].dtype=="float64":
        print(cols,i)
#         i+=1
print(i)

MSSubClass 0
LotFrontage 1
LotArea 1
OverallQual 12
OverallCond 12
YearBuilt 12
YearRemodAdd 12
BsmtFinSF1 24
BsmtUnfSF 25
TotalBsmtSF 25
1stFlrSF 29
GrLivArea 29
FullBath 29
BedroomAbvGr 29
KitchenAbvGr 29
TotRmsAbvGrd 30
Fireplaces 31
GarageYrBlt 33
GarageCars 34
GarageArea 34
OpenPorchSF 37
MoSold 37
YrSold 37
39


In [110]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [111]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [112]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [113]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [114]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [115]:
continuous_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'FullBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'OpenPorchSF',
 'MoSold',
 'YrSold']

In [116]:
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [117]:

def simplify_skewed_columns(df_train:pd.DataFrame,strategy:str,threshold:int):
    most_skewed=[]
    for col in continuous_cols:
        if df_train[col].skew()<-threshold or df_train[col].skew()>threshold:
            most_skewed.append(col)

          
    # Removing skewness in columns

    for col in most_skewed:

        if strategy=="boxcox":
            df_train[col],parameters=stat.boxcox(df_train[col])
        elif strategy=="log":
            trf = FunctionTransformer(func=np.log1p)
            X=trf.fit_transform(df_train[col])
            df_train[col]=X
        elif strategy=="sqrt":
            df_train[col]=np.sqrt(df_train[col])

In [118]:
simplify_skewed_columns(df_train=df_train,strategy="log",threshold=1)

In [119]:
for col in continuous_cols:
    if df_train[col].skew()<-1 or df_train[col].skew()>1:
        print(col)
        # df_train.drop(col,axis=1,inplace=True)

TotalBsmtSF
KitchenAbvGr


In [120]:
df_train.loc[:,continuous_cols]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
0,4.110874,4.189655,9.042040,7,5,2003,2003,6.561031,150,6.753438,...,3,0.693147,8,0,2003.0,2,548,4.127134,2,2008
1,3.044522,4.394449,9.169623,6,8,1976,1976,6.886532,284,7.141245,...,3,0.693147,6,1,1976.0,2,460,0.000000,5,2007
2,4.110874,4.234107,9.328212,7,5,2001,2002,6.188264,434,6.825460,...,3,0.693147,6,1,2001.0,2,608,3.761200,9,2008
3,4.262680,4.110874,9.164401,7,5,1915,1970,5.379897,540,6.629363,...,3,0.693147,7,1,1998.0,3,642,3.583519,2,2006
4,4.110874,4.442651,9.565284,8,5,2000,2000,6.486161,490,7.044033,...,4,0.693147,9,1,2000.0,3,836,4.442651,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,4.110874,4.143135,8.976894,6,5,1999,2000,0.000000,953,6.860664,...,3,0.693147,7,1,1999.0,2,460,3.713572,8,2007
1456,3.044522,4.454347,9.486152,6,6,1978,1988,6.673298,589,7.341484,...,3,0.693147,7,2,1978.0,2,500,0.000000,2,2010
1457,4.262680,4.204693,9.109746,7,9,1941,2006,5.620401,877,7.050123,...,4,0.693147,9,2,1941.0,1,252,4.110874,5,2010
1458,3.044522,4.234107,9.181735,5,6,1950,1996,3.912023,0,6.983790,...,2,0.693147,5,0,1950.0,1,240,0.000000,4,2010


In [121]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(20, 20)

# for ax, col in zip(axes, continuous_cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [123]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.loc[:,continuous_cols].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [124]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here


In [125]:
# Outlier delete

for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)] 
    df_train = df_train[(df_train[col] >=l)] 



[ 1.2172862787465843 , 6.089916036018154 ]
[ 3.7041447084224566 , 4.7887557904247355 ]
[ 8.401753817140381 , 9.933102238156422 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1886.0 , 2070.0 ]
[ 1908.0 , 2060.0 ]
[ -9.897763542921773 , 16.496272571536288 ]
[ -614.25 , 1675.75 ]
[ 6.057197377166374 , 7.800201976129094 ]
[ 6.201413723806424 , 7.836424853015848 ]
[ 6.374620590969741 , 8.157392124352672 ]
[ -0.5 , 3.5 ]
[ 3.0 , 3.0 ]
[ 0.6931471805599453 , 0.6931471805599453 ]
[ 4.5 , 8.5 ]
[ -1.5 , 2.5 ]
[ 1908.5 , 2056.5 ]
[ -0.5 , 3.5 ]
[ 23.5 , 907.5 ]
[ -6.394019815561974 , 10.65669969260329 ]
[ 0.5 , 12.5 ]
[ 2004.0 , 2012.0 ]


In [126]:
output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

In [128]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train
        

In [129]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="labelencoder")

In [130]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
count,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,...,579.0,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000
mean,3.585684,4.280895,9.193766,6.157168,5.493955,1977.219344,1984.955095,4.521871,574.664940,6.955427,...,3.0,0.693147,6.438687,0.623489,1980.469775,1.830743,489.540587,2.410905,6.267703,2007.813472
std,0.572747,0.171136,0.237506,1.124359,0.778972,25.315693,19.826533,2.873593,414.489341,0.285947,...,0.0,0.000000,0.898454,0.605512,22.831572,0.584418,163.112576,2.108245,2.697399,1.338707
min,3.044522,3.713572,8.412055,3.000000,4.000000,1900.000000,1950.000000,0.000000,0.000000,6.228511,...,3.0,0.693147,5.000000,0.000000,1916.000000,1.000000,180.000000,0.000000,1.000000,2006.000000
25%,3.044522,4.189655,9.050875,5.000000,5.000000,1960.000000,1967.000000,0.000000,253.500000,6.762730,...,3.0,0.693147,6.000000,0.000000,1964.000000,1.000000,384.000000,0.000000,5.000000,2007.000000
50%,3.044522,4.262680,9.189219,6.000000,5.000000,1977.000000,1993.000000,6.126869,484.000000,6.947937,...,3.0,0.693147,6.000000,1.000000,1981.000000,2.000000,484.000000,3.433987,6.000000,2008.000000
75%,4.110874,4.394449,9.332159,7.000000,6.000000,2001.000000,2003.000000,6.617402,803.500000,7.169735,...,3.0,0.693147,7.000000,1.000000,2002.000000,2.000000,576.000000,4.262680,8.000000,2009.000000
max,5.252273,4.779123,9.930373,10.000000,7.000000,2009.000000,2010.000000,7.539559,1670.000000,7.707063,...,3.0,0.693147,8.000000,2.000000,2010.000000,3.000000,905.000000,6.037871,12.000000,2010.000000


In [131]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>0:
        print(col)

In [132]:
### SCALING

In [133]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.483003,0.446795,0.414923,0.571429,0.333333,0.944954,0.883333,0.870214,0.089820,0.355028,...,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
1,0.483003,0.488512,0.603403,0.571429,0.333333,0.926606,0.866667,0.820773,0.259880,0.403739,...,0.333333,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
2,0.551764,0.372860,0.495513,0.571429,0.333333,0.137615,0.333333,0.713556,0.323353,0.271111,...,0.333333,0.000000,0.00,0.2,0.5,0.0,0.0,0.0,0.000000,0.50
3,0.000000,0.579194,0.531344,0.714286,0.333333,0.954128,0.916667,0.957956,0.189820,0.813090,...,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
4,0.483003,0.502015,0.550524,0.571429,0.666667,0.669725,0.383333,0.896197,0.129341,0.528761,...,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,0.483003,0.638990,0.525835,0.714286,0.333333,0.871560,0.766667,0.926409,0.084431,0.594443,...,0.333333,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
575,0.000000,0.615527,0.475347,0.714286,0.333333,0.990826,0.983333,0.000000,0.941916,0.766199,...,0.666667,0.000000,0.00,0.0,1.0,0.0,0.0,0.0,0.142857,0.25
576,0.483003,0.403136,0.372016,0.428571,0.333333,0.908257,0.833333,0.000000,0.570659,0.427549,...,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
577,0.000000,0.695204,0.707426,0.428571,0.666667,0.715596,0.633333,0.885105,0.352695,0.752745,...,0.000000,0.333333,0.25,0.0,0.5,0.0,0.0,0.0,0.000000,0.00


In [134]:
from sklearn.metrics import r2_score

In [135]:
X=df_train
Y=output_col

In [136]:
accuracies={}

In [137]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


In [138]:
def sf_selector(algo:str,forward:bool,floating:bool,scoring:int,X_train,Y_train,X_test,Y_test,min_featues:int,cv:int,verbose:int):
    model=eval(algo)
    sfs=SFS(model, 
           k_features=(min_featues,len(X_train.columns)), 
           forward=forward, 
           floating=floating, 
           verbose=verbose,
           scoring=scoring,
           cv=cv,
           n_jobs=-1)
    sfs=sfs.fit(X_train, Y_train)
    print("Selected Features",sfs.k_feature_names_)
    model_=eval(algo)
    model_.fit(X_train.loc[:,list(sfs.k_feature_names_)],Y_train)
    score=model_.score(X_test.loc[:,list(sfs.k_feature_names_)],Y_test)
    
    print(f"Prediction using selected features using {model_} is :",score)
    accuracies[model]=score

In [ ]:
sf_selector(algo="LinearRegression()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [141]:
sf_selector(algo="RandomForestRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    5.5s finished

[2023-01-29 12:46:40] Features: 1/62 -- score: 0.6739927479405168[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    5.9s finished

[2023-01-29 12:46:46] Features: 2/62 -- score: 0.7186654773692046[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    5.8s finished

[2023-01-29 12:46:52] Features: 3/62 -- score: 0.7384999748029251[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  59 out o

Selected Features ('OverallQual', 'OverallCond', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 'GarageCars', 'MSZoning', 'LotShape', 'Utilities', 'MasVnrType', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtExposure', 'Heating', 'KitchenQual', 'SaleCondition')
Prediction using selected features using RandomForestRegressor() is : 0.8911745072572475


In [158]:
sf_selector(algo="SVR()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    1.5s finished

[2023-01-29 15:18:16] Features: 1/62 -- score: -0.02733723438192066[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.5s finished

[2023-01-29 15:18:16] Features: 2/62 -- score: -0.02747586098739028[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished

[2023-01-29 15:18:17] Features: 3/62 -- score: -0.02754275353014357[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.6s finished

[2023-01-29 15:18:17] Features: 4/62 -- score: -0.027585725606625467[Parallel(n_job

Selected Features ('FullBath', 'BedroomAbvGr', 'ExterQual')
Prediction using selected features using SVR() is : -0.01428189208687991


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s finished

[2023-01-29 15:18:37] Features: 61/62 -- score: -0.02956577668126652[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished

[2023-01-29 15:18:37] Features: 62/62 -- score: -0.029614107780512544

In [3]:
# sf_selector(algo="KNeighborsRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [4]:
# sf_selector(algo="DecisionTreeRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [5]:
# sf_selector(algo="SGDRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [6]:
# sf_selector(algo="ElasticNet()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [7]:
# sf_selector(algo="Lasso()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [8]:
# sf_selector(algo="Ridge()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [183]:
accuracies

{LinearRegression(): 0.8931279482412343,
 RandomForestRegressor(): 0.8911745072572475,
 SVR(): -0.01428189208687991,
 KNeighborsRegressor(): 0.8507376965311726,
 DecisionTreeRegressor(): 0.8176195807148101,
 SGDRegressor(): 0.8917351605621597,
 ElasticNet(): 0.6515002257408309,
 Lasso(): 0.8931842292077178,
 Ridge(): 0.8893798372352621}